In [395]:
from sklearn.datasets import load_boston
from sklearn.metrics import explained_variance_score
import numpy as np

x = load_boston().data
y = load_boston().target

In [396]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=29)

x_train.shape

(379, 13)

In [525]:
def polynom(m, deg):
    n_rows, n_cols = m.shape[0], m.shape[1]
    nm = np.zeros((m.shape[0],n_cols*deg))
    for d in range(deg):
        for c in range(n_cols):
            for r in range(n_rows):
                nm[r][n_cols*d+c] = m[r][c]**(d+1)
    return nm

In [543]:
# scaling
def scaling(X):
    for c in range(X.shape[1]):
        X[:,c] = [(i - np.mean(X[:,c])) / (max(X[:,c]) - min(X[:,c])) for i in X[:,c]]
    return X

# normalize
def normal(X):
    return X / X.max(axis=0)

normal(x)

matrix([[-1. , -1. , -0.5, -0.5, -0.5],
        [ 0. ,  0. , -0.5,  1. , -0.5],
        [ 1. ,  1. ,  1. , -0.5,  1. ]])

In [544]:
def predict(X, theta):
    return np.c_[np.ones(len(X)), normal(X)] @ theta

In [522]:
def normalEq(X, y):
    X = np.c_[np.ones(len(X)), scaling(X)]
    return (np.linalg.inv(X.T @ X) @ X.T @ y).T

theta = normalEq(x_train, y_train)

print(theta)

preds = predict(x_test, theta)

print(f'\nNorm Eq EVS: {explained_variance_score(preds, y_test)}')

[ 22.61794195  -6.88081299   5.31800698   1.00151806   3.43338847
  -7.19704237  18.47655796  -0.2396351  -15.02450125   7.41163346
  -6.77932357  -9.07260822   4.38736543 -19.97598678]

Norm Eq EVS: 0.7416264097926808


$X^{'}X=\begin{bmatrix} 7 & 38.5\\  38.5& 218.75 \end{bmatrix}$

In [545]:
def BGD(X, y, n_iter, eta):
    X = np.c_[np.ones(len(X)), normal(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    for i in range(n_iter):
        for c in range(X.shape[1]):
            nX = X[:,c]
            nT = theta[:,c]
            gradients = ((X @ theta.T) - y.T).T @ nX
            theta[:,c] = nT - eta * gradients
    return theta.T

theta = BGD(polynom(x_train, 2), y_train, 1000, 0.001)

print(theta)

preds = predict(polynom(x_test, 2), theta)

print(f'\nBGD EVS: {explained_variance_score(preds, y_test)}')

[[ 19.20634228]
 [ -9.2427743 ]
 [ -2.17089119]
 [  0.66995084]
 [  0.94126055]
 [ -5.02641502]
 [  5.73495914]
 [  0.60629257]
 [ -8.35755114]
 [  5.7156615 ]
 [ -4.16123053]
 [ -2.36057202]
 [  0.11475703]
 [-20.90252573]
 [ -0.87968749]
 [  5.11832911]
 [  2.25772547]
 [  2.21811194]
 [ -1.24343535]
 [ 13.15609598]
 [  2.53579219]
 [  2.55018273]
 [ -1.9074604 ]
 [  0.75192262]
 [  3.72083115]
 [ -2.47023228]
 [ 14.19620019]]

BGD EVS: 0.8140623442564865


In [546]:
def learning_schedule(t):
    return 5 / (t + 50)

def SGD(X, y, n_epoch, eta):
    X = np.c_[np.ones(len(X)), normal(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    m = X.shape[0]
    for epoch in range(1, n_epoch):
        for n in range(m):
            for c in range(X.shape[1]):
                r = np.random.randint(0, m-1)
                xi, yi = X[r:r+1], y.T[r:r+1]
                gradients = ((xi @ theta.T) - yi.T).T @ xi[:,c]
#                 eta = eta/epoch
                theta[:,c] = theta[:,c] - (gradients * eta)
    return theta.T

theta = SGD(polynom(x_train, 2), y_train, 50, 0.01)

print(theta)

preds = predict(polynom(x_test, 2), theta)

print(f'\nSGD EVS: {explained_variance_score(preds, y_test)}')

[[ 18.83559704]
 [ -7.47011212]
 [ -1.08192989]
 [  0.22937703]
 [ -0.429755  ]
 [ -3.26909609]
 [  7.26167115]
 [  0.58524702]
 [ -6.9499567 ]
 [  4.96951045]
 [ -4.60556924]
 [ -2.29254982]
 [  0.33788698]
 [-18.3234577 ]
 [ -1.24059598]
 [  3.93647342]
 [  4.72796103]
 [  2.67272805]
 [ -2.46314802]
 [ 14.66677497]
 [  3.24139179]
 [  1.01073016]
 [ -0.2324581 ]
 [  0.08740002]
 [  2.30703379]
 [  0.11071134]
 [ 10.52767917]]

SGD EVS: 0.827425550330545


In [547]:
def MBGD(X, y, n_epoch, batch_size, eta):
    X = np.c_[np.ones(len(X)), normal(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    m = X.shape[0]
    for epoch in range(n_epoch):
        shuffi = np.random.permutation(m)
        xs, ys = X[shuffi], y.T[shuffi]
        for i in range(0, m, batch_size):
            for c in range(X.shape[1]):
                r = np.random.randint(0, m-1)
                xi, yi = X[i:i+batch_size], y.T[i:i+batch_size]
                gradients = ((xi @ theta.T) - yi).T @ xi[:,c]
                theta[:,c] = theta[:,c] - (gradients * eta)
    return theta.T
        
theta = MBGD(polynom(x_train, 2), y_train, 100, 20, 0.01)

print(theta)

preds = predict(polynom(x_test, 2), theta)

print(f'\n MBGD EVS: {explained_variance_score(preds, y_test)}')

[[ 22.53683872]
 [ -5.51904959]
 [ -3.4627349 ]
 [  0.83913856]
 [ -0.60705795]
 [ -7.93832232]
 [  6.66307182]
 [  0.75006893]
 [-10.00536549]
 [  9.5157487 ]
 [ -4.18323917]
 [ -1.45006426]
 [ -1.30930688]
 [-20.72920014]
 [ -6.27117239]
 [  7.10618789]
 [  3.14753082]
 [  3.70272644]
 [  0.41509772]
 [ 12.27888272]
 [  1.19565218]
 [  2.81536288]
 [ -6.93870558]
 [  0.26005536]
 [  5.20769191]
 [-24.5679934 ]
 [ 14.95062064]]

 MBGD EVS: 0.796540884604747


In [454]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

polybig_features = PolynomialFeatures(degree=2, include_bias=False)
std_scaler = StandardScaler()
lin_reg = LinearRegression()
polynomial_regression = Pipeline([
        ("poly_features", polybig_features),
        ("std_scaler", std_scaler),
        ("lin_reg", lin_reg),
    ])
polynomial_regression.fit(x_train, y_train)
preds = polynomial_regression.predict(x_test)

In [455]:
m = LinearRegression()

m.fit(polynom(x_train, 2), y_train)

preds = m.predict(polynom(x_test, 2))

print('weights: ')
print(m.coef_)
print('Intercept: ')
print(m.intercept_)

print(f'\n Scikit learn lin reg EVS: {explained_variance_score(y_test, preds)}')

weights: 
[ -4.82353170e-01  -6.11811058e-02  -1.03006200e-01   1.52751036e+00
  -2.66093614e+01  -1.72065010e+01  -5.67860772e-02  -2.20943742e+00
   5.80765699e-01  -2.01868784e-02  -6.10801742e+00   2.14283984e-02
  -1.54975811e+00   4.96884001e-03   8.07538384e-04   6.18487450e-03
   1.52751037e+00   3.81582232e+00   1.54888132e+00   5.40426439e-04
   1.03788287e-01  -4.23190591e-03   6.78223968e-06   1.49536968e-01
  -3.09227294e-05   2.80283423e-02]
Intercept: 
164.391664059
Scikit learn lin reg EVS: 0.8153831306538188
